<a href="https://colab.research.google.com/github/jhkr1/BoostCorse-AITech-Pre-Course/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. PyTorch 모델 저장하기 & 불러오기

In [1]:
!pip install torchsummary

In [2]:
import os
import shutil
import zipfile
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchsummary import summary
from os import walk
from PIL import Image
from torchvision import datasets
from torchvision import models

warnings.filterwarnings('ignore')

In [3]:
# 신경망 모델 클래스 정의
# TheModelClass라는 설계도 만들기
# nn.Module를 기본 설계도를 상속 받아서 -> nn.Module에는 학습에 필요한 모든 기능이 구현 되어있음.
class TheModelClass(nn.Module):
  def __init__(self):
    super(TheModelClass, self).__init__()

    # 첫 번째 층: 합성곱, 배치 정규화, ReLU 활성화 함수, 최대 풀링을 포함
    # Sequential -> 컨테이너 역할
    self.layer1 = nn.Sequential(
        # 이미지의 특징을 추출
        nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=0),
        # 배치 정규화 -> 평균 0, 분산1 -> 학습이 더 안정적이고, 빠르게 하기 위해
        nn.BatchNorm2d(16),
        # 활성화함수 0보다 작은 값 -> 0, 0보다 큰 값은 그대로
        nn.ReLU(),
        # 이미지의 크기를 줄여 계산량 감소, 가장 중요한 특징만 남기기.
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    # 두 번째 층
    self.layer2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=0),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    # 세 번째 층
    self.layer3 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    # 드롭아웃 층
    # 과적합을 막기 위한 장치
    # 학습 과정에서 일부로 몇몇 연결을 끊어, 특정 연결에 의존 해결
    self.drop_out = nn.Dropout()

    # 완전 연결 층
    # 특징들을 종합하여 분류를 준비
    # 3 * 3 * 64는 layer3을 통과한 feature map을 1차원으로 펼친 크기
    self.fc1 = nn.Linear(3 * 3 * 64, 1000)
    # 최종 출력 만들기.
    self.fc2 = nn.Linear(1000, 1)

  # 순전파 함수 정의
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)

    # 텐서를 1차원으로 변환
    # out.size(0)은 배치 크기 유지, -1은 나머지를 모두 1차원으로 만들라는 의미
    out = out.view(out.size(0), -1)
    out = self.drop_out(out)
    out = self.fc1(out)
    out = self.fc2(out)

    return out

In [4]:
# 모델 초기화 및 GPU로 이동
model = TheModelClass().cuda()

# 옵티마이저 초기화:SGD 사용
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [6]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
  print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
layer1.0.weight 	 torch.Size([16, 3, 3, 3])
layer1.0.bias 	 torch.Size([16])
layer1.1.weight 	 torch.Size([16])
layer1.1.bias 	 torch.Size([16])
layer1.1.running_mean 	 torch.Size([16])
layer1.1.running_var 	 torch.Size([16])
layer1.1.num_batches_tracked 	 torch.Size([])
layer2.0.weight 	 torch.Size([32, 16, 3, 3])
layer2.0.bias 	 torch.Size([32])
layer2.1.weight 	 torch.Size([32])
layer2.1.bias 	 torch.Size([32])
layer2.1.running_mean 	 torch.Size([32])
layer2.1.running_var 	 torch.Size([32])
layer2.1.num_batches_tracked 	 torch.Size([])
layer3.0.weight 	 torch.Size([64, 32, 3, 3])
layer3.0.bias 	 torch.Size([64])
layer3.1.weight 	 torch.Size([64])
layer3.1.bias 	 torch.Size([64])
layer3.1.running_mean 	 torch.Size([64])
layer3.1.running_var 	 torch.Size([64])
layer3.1.num_batches_tracked 	 torch.Size([])
fc1.weight 	 torch.Size([1000, 576])
fc1.bias 	 torch.Size([1000])
fc2.weight 	 torch.Size([1, 1000])
fc2.bias 	 torch.Size([1])


In [7]:
# 모델의 상태 사전의 타입 출력
type(model.state_dict())

collections.OrderedDict

In [8]:
# 모델의 구조 및 파라미터 요약 출력
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 111, 111]             448
       BatchNorm2d-2         [-1, 16, 111, 111]              32
              ReLU-3         [-1, 16, 111, 111]               0
         MaxPool2d-4           [-1, 16, 55, 55]               0
            Conv2d-5           [-1, 32, 27, 27]           4,640
       BatchNorm2d-6           [-1, 32, 27, 27]              64
              ReLU-7           [-1, 32, 27, 27]               0
         MaxPool2d-8           [-1, 32, 13, 13]               0
            Conv2d-9             [-1, 64, 6, 6]          18,496
      BatchNorm2d-10             [-1, 64, 6, 6]             128
             ReLU-11             [-1, 64, 6, 6]               0
        MaxPool2d-12             [-1, 64, 3, 3]               0
          Dropout-13                  [-1, 576]               0
           Linear-14                 [-

In [9]:
# 모델을 저장할 경로를 지정
MODEL_PATH = 'saved'

# 지정된 경로가 존재하지 않으면 해당 경로를 생성
if not os.path.exists(MODEL_PATH):
  os.makedirs(MODEL_PATH)

# 모델의 상태 사전(state_dict)를 저장한다.
torch.save(model.state_dict(), os.path.join(MODEL_PATH, 'model.pt'))

In [13]:
# GPU를 사용 가능하다면 cuda를, 아니라면 cpu 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 새로운 모델 객체 생성
new_model = TheModelClass()

# 저장된 상태 사전을 불러와 새로운 모델에 적용
new_model.load_state_dict(torch.load(os.path.join(MODEL_PATH, 'model.pt')))

<All keys matched successfully>

In [15]:
# 전체 모델을 파일로 저장합니다.
torch.save(model, os.path.join(MODEL_PATH, "model_pickle.pt"))

# 저장된 전체 모델을 불러옵니다.
model = torch.load(os.path.join(MODEL_PATH, "model_pickle.pt"), weights_only=False)

# 모델을 평가 모드로 설정합니다. (드롭아웃 및 배치 정규화를 비활성화)
model.eval()

TheModelClass(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=576, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=1, b

In [16]:
# 개와 고양이 데이터셋 가져오기
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

--2025-07-20 06:04:51--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.218.186.12, 2600:1409:3c00:c80::317f, 2600:1409:3c00:c8c::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.218.186.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M   130MB/s    in 11s     

2025-07-20 06:05:02 (72.3 MB/s) - ‘kagglecatsanddogs_5340.zip’ saved [824887076/824887076]



In [17]:
# 압축 파일명을 지정한다.
filename = "kagglecatsanddogs_5340.zip"

# 압축 파일을 해제한다.
with zipfile.ZipFile(filename, 'r') as zip_ref:
  zip_ref.extractall()

# 'PetImages' 폴더를 'data'폴더로 이동한다.
shutil.move('PetImages', 'data')

# 'data' 폴더 내의 모든 파일 경로를 가져온다.
mypath = "data"
f_path = []

for (dirpath, dirnames, filenames) in walk(mypath):
  f_path.extend([os.path.join(dirpath, filename) for filename in filenames])

# 손상된 이미지 파일이 있는지 확인하고, 손상된 파일을 삭제
for f_name in f_path:
  try:
    Image.open(f_name)
  except Exception as e:
    print(e)
    os.remove(f_name)

cannot identify image file 'data/Cat/Thumbs.db'
cannot identify image file 'data/Cat/666.jpg'
cannot identify image file 'data/Dog/Thumbs.db'
cannot identify image file 'data/Dog/11702.jpg'


In [20]:
# 데이터셋을 로드하고 전처리를 적용한다.
# datasets.ImageFolde에서 root = 'data'로 지정하면 아래와 같은 구조라고 가정
# data/
# ├── class_A/
# │   ├── xxx.png
# │   ├── xxy.png
# │   └── ...
# │
# └── class_B/
#     ├── zzz.png
#     ├── zzy.png
#     └── ...

dataset = datasets.ImageFolder(root="data",
                               # Compose는 이미지 천저리(transform)를 하나의 작업으로 묶어주는 역할
                               transform=transforms.Compose([
                                   transforms.Resize(224),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5),
                                                        (0.5, 0.5, 0.5))
                               ]))

# DataLoader는 Dataset을 감싸서, 학습에 최적화된 방식으로 데이터를 공급해주는 관리자
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=8,
                                         shuffle=True,
                                         # 8개의 CPU 프로세스를 사용해 병렬 처리 -> 병목 현상 감소
                                         num_workers=8)

In [22]:
# 이진 분류 정확도를 계산하는 함수
# y_pred: 모델이 출력한 원시 값(logits)
# y_test: 실제 정답 라벨
def binary_acc(y_pred, y_test):

   # 1. 시그모이드(sigmoid) 함수를 적용하여 모델의 출력을 0과 1 사이의 확률 값으로 변환하고,
   #    round 함수를 이용해 0.5를 기준으로 예측을 0 또는 1로 결정합니다.
  y_pred_tag = torch.round(torch.sigmoid(y_pred))

  # 2. 예측 결과(y_pred_tag)와 실제 정답(y_test)이 일치하는 개수를 셉니다.
  #    (y_pred_tag == y_test)는 [True, False, True, ...] 형태의 불리언 텐서를 반환하고,
  #    .sum()은 True의 개수를 합산합니다. .float()은 나눗셈을 위해 실수로 변환합니다.
  correct_results_sum = (y_pred_tag == y_test).sum().float()

  # 3. (정답 개수 / 전체 데이터 개수)로 정확도를 계산합니다.
  acc = correct_results_sum / y_test.shape[0]

  # 4. 정확도를 백분율로 변환하고 소수점을 반올림하여 정수로 만듭니다.
  acc = torch.round(acc * 100)

  return acc

In [23]:
# 학습 설정값을 지정
EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.1

In [24]:
# 모델을 GPU로 이동한다.
model.to(device)

# 손실 함수와 옵티마이저를 설정
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [25]:
# 학습을 시작합니다.
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.FloatTensor)

        optimizer.zero_grad()
        y_pred = model(X_batch)

        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    # 모델의 상태와 옵티마이저의 상태를 저장합니다.
    torch.save({
        'epoch': e,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
        }, f"saved/checkpoint_model_{e}_{epoch_loss/len(dataloader)}_{epoch_acc/len(dataloader)}.pt")

    # 에폭별 손실과 정확도를 출력합니다.
    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(dataloader):.5f} | Acc: {epoch_acc/len(dataloader):.3f}')

Epoch 001: | Loss: 11.03036 | Acc: 49.596
Epoch 002: | Loss: 1.14812 | Acc: 49.889
Epoch 003: | Loss: 0.70050 | Acc: 49.798
Epoch 004: | Loss: 0.70304 | Acc: 50.212
Epoch 005: | Loss: 0.80617 | Acc: 50.371
Epoch 006: | Loss: 0.92092 | Acc: 50.086
Epoch 007: | Loss: 0.85282 | Acc: 50.702
Epoch 008: | Loss: 0.88216 | Acc: 50.377


KeyboardInterrupt: 

## 2. Pretrained Model 불러오기

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 사전 학습된 VGG16 모델 불러오기
vgg = models.vgg16(pretrained=True).to(device)

# VGG16 모델의 구조 출력
print(vgg)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 77.5MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [27]:
# VGG16 모델의 구조 및 파라미터 요약을 출력한다.
summary(vgg, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [28]:
# VGG16 모델의 각 층의 이름과 구조를 출력
for name, layer in vgg.named_modules():
  print(name, layer)

 VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [29]:
# VGG 모델의 완전 연결 층(fc)을 수정하여 출력 유닛의 수를 1로 변경
vgg.fc = torch.nn.Linear(1000, 1)

# 모델을 GPU로 이동
vgg.cuda()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [30]:
# 다시 사전 학습된 VGG16 모델을 불러오기
vgg = models.vgg16(pretrained=True).to(device)

# vgg16 모델의 구조 출력
print(vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [32]:
# VGG16 모델의 분류기(classifier) 중 마지막 선형 층을 수정하여 출력 유닛의 수를 1로 변경
vgg.classifier._modules['6'] = torch.nn.Linear(4096, 1)

# 모델을 GPU로 이동
vgg.cuda()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## 3. Pretrained Model로 나만의 모델 만들기

In [33]:
# 'data/' 경로의 이미지 데이터를 불러오고 전처리를 적용
dataset = datasets.ImageFolder(root ="data/",
                               transform=transforms.Compose([
                                   transforms.Resize(224),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5),
                                                        (0.5, 0.5, 0.5))
                               ]))
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=8,
    shuffle=True,
    num_workers=8
)

In [34]:
# 새로운 신경망 모델을 정의
class MyNewNet(nn.Module):
    def __init__(self):
        super(MyNewNet, self).__init__()
        self.vgg19 = models.vgg19(pretrained=True)
        self.linear_layers = nn.Linear(1000, 1) # 추가적인 선형 층을 정의

    def forward(self, x):
      x = self.vgg19(x)
      return self.linear_layers(x)

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델을 초기화하고 GPU로 이동
my_model = MyNewNet()
my_model = my_model.to(device)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:05<00:00, 114MB/s]


In [36]:
# 모델의 구조를 출력
print(my_model)

MyNewNet(
  (vgg19): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [37]:
# 모델의 모든 파라미터를 고정
for param in my_model.parameters():
  param.requires_grad = False

In [38]:
# 선형 층의 파라미터만 학습 가능하도록 설정
for param in my_model.linear_layers.parameters():
  param.requires_grad = True

In [39]:
my_model.eval()

MyNewNet(
  (vgg19): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [41]:
# 데이터 로더에서 첫 번째 배치를 가져오기
x = next(iter(dataloader))

[tensor([[[[-0.9765, -0.9765, -0.8902,  ..., -0.9216, -0.8196, -0.7255],
           [-0.9608, -0.9765, -0.8667,  ..., -0.8980, -0.7882, -0.7333],
           [-0.9451, -0.9686, -0.8510,  ..., -0.9059, -0.8353, -0.8275],
           ...,
           [-0.7255, -0.7333, -0.7490,  ..., -0.8510, -0.8510, -0.8510],
           [-0.7255, -0.7333, -0.7255,  ..., -0.8431, -0.8353, -0.8431],
           [-0.7255, -0.7333, -0.7176,  ..., -0.8510, -0.8275, -0.8275]],
 
          [[-0.9686, -0.9529, -0.8667,  ..., -0.9843, -0.8902, -0.8039],
           [-0.9529, -0.9529, -0.8431,  ..., -0.9686, -0.8667, -0.8118],
           [-0.9373, -0.9451, -0.8275,  ..., -0.9686, -0.8980, -0.8902],
           ...,
           [-0.8667, -0.8824, -0.8980,  ..., -0.8902, -0.8902, -0.8902],
           [-0.8667, -0.8824, -0.8745,  ..., -0.8824, -0.8745, -0.8824],
           [-0.8667, -0.8745, -0.8667,  ..., -0.8902, -0.8667, -0.8667]],
 
          [[-0.9765, -0.9608, -0.8745,  ..., -0.9922, -0.8980, -0.8118],
           [-

In [44]:
# 입력 데이터의 형태를 출력
x[0].shape

torch.Size([8, 3, 224, 224])

In [45]:
my_model(x[0].to(device))

tensor([[-0.5986],
        [ 0.8094],
        [ 2.2728],
        [-0.5023],
        [-0.5125],
        [-0.5517],
        [ 1.8294],
        [ 1.2925]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [46]:
# 학습 설정값을 지정합니다.
EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [47]:
# 데이터 로더의 배치 크기를 업데이트합니다.
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=BATCH_SIZE,
                                         shuffle=True,
                                         num_workers=8)

In [48]:
# VGG19 모델의 첫 번째 합성곱 층의 파라미터를 출력합니다.
next(my_model.vgg19.features._modules['0'].parameters())[0]

tensor([[[-0.0535, -0.0493, -0.0679],
         [ 0.0153,  0.0451,  0.0021],
         [ 0.0362,  0.0200,  0.0199]],

        [[ 0.0170,  0.0554, -0.0062],
         [ 0.1416,  0.2271,  0.1376],
         [ 0.1200,  0.2003,  0.0921]],

        [[-0.0449,  0.0127, -0.0145],
         [ 0.0597,  0.1395,  0.0541],
         [-0.0010,  0.0583, -0.0297]]], device='cuda:0')

In [49]:
# 선형 층의 파라미터 중 첫 10개를 출력합니다.
it = my_model.linear_layers.parameters()
next(it)[0][:10]

tensor([-0.0239,  0.0237,  0.0030, -0.0092, -0.0102, -0.0170,  0.0132,  0.0123,
         0.0113,  0.0009], device='cuda:0', grad_fn=<SliceBackward0>)

In [50]:
# 이진 분류 정확돌르 계산하는 함수를 정의
def binary_acc(y_pred, y_test):
  y_pred_tag = torch.round(torch.sigmoid(y_pred))
  correct_results_sum = (y_pred_tag == y_test).sum().float()
  acc = correct_results_sum / y_test.shape[0]
  acc = torch.round(acc * 100)

  return acc

# 모델을 초기화하고 GPU로 이동
my_model = MyNewNet()
my_model = my_model.to(device)

# 모델의 모든 파라미터를 고정
for param in my_model.parameters():
  param.requires_grad = False

# 선형 층의 파라미턴 학습 가능하도록 설정
for param in my_model.linear_layers.parameters():
  param.requires_grad = True

# 손실 함수와 옵티마이저 설정
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(my_model.parameters(), lr=LEARNING_RATE)

# 학습을 시작합니다.
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.FloatTensor)

        optimizer.zero_grad()
        y_pred = my_model(X_batch)

        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    # 에폭별 손실과 정확도를 출력합니다.
    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(dataloader):.5f} | Acc: {epoch_acc/len(dataloader):.3f}')

Epoch 001: | Loss: 0.06804 | Acc: 97.182


KeyboardInterrupt: 

In [51]:
# VGG19 모델의 첫 번째 합성곱 층의 파라미터를 출력합니다.
next(my_model.vgg19.features._modules['0'].parameters())[0]

tensor([[[-0.0535, -0.0493, -0.0679],
         [ 0.0153,  0.0451,  0.0021],
         [ 0.0362,  0.0200,  0.0199]],

        [[ 0.0170,  0.0554, -0.0062],
         [ 0.1416,  0.2271,  0.1376],
         [ 0.1200,  0.2003,  0.0921]],

        [[-0.0449,  0.0127, -0.0145],
         [ 0.0597,  0.1395,  0.0541],
         [-0.0010,  0.0583, -0.0297]]], device='cuda:0')

In [52]:
# 선형 층의 파라미터 중 첫 10개를 출력합니다.
it = my_model.linear_layers.parameters()
next(it)[0][:10]

#  tensor([-0.0239,  0.0237,  0.0030, -0.0092, -0.0102, -0.0170,  0.0132,  0.0123,
#          0.0113,  0.0009], device='cuda:0', grad_fn=<SliceBackward0>)

tensor([-0.0209, -0.0484,  0.0068, -0.0317, -0.0138, -0.0225,  0.0195, -0.0146,
         0.0019,  0.0030], device='cuda:0', grad_fn=<SliceBackward0>)